In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import exp,log,mean
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from numpy import log,dot,exp,shape,min,max,clip,mean,sqrt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  
from tensorflow.keras import utils as np_utils
from keras.utils import to_categorical
import random
# np.random.set_state(42)
random_state = 42
np.random.state = random_state
np.random.seed = random_state

# Car Price Prediction 
```
(Regression problem [first dataset])
```


In [ ]:
car_price=pd.read_csv('Car Price Prediction.csv')
print(car_price.head())
print(car_price.info())
print(car_price.describe())
print(car_price.isna().any())

In [ ]:
car_price.head()


In [ ]:
#tranform object type cols to int
print(car_price.Fuel_Type.value_counts())
print(car_price.Seller_Type.value_counts())
print(car_price.Transmission.value_counts())

In [ ]:
car_price.Fuel_Type.replace({"Petrol":"0","Diesel":"1","CNG":"2"},inplace=True)
car_price.Seller_Type.replace({"Dealer":"0","Individual":"1"},inplace=True)
car_price.Transmission.replace({"Manual":"0","Automatic":"1"},inplace=True)
car_price[["Fuel_Type","Seller_Type","Transmission"]]=car_price[["Fuel_Type","Seller_Type","Transmission"]].astype(int)

In [ ]:
y_car_price=car_price.Selling_Price
X_car_price=car_price.drop(["Selling_Price","Car_Name"],axis=1)
X_car_price=np.array(X_car_price)
y_car_price=np.array(y_car_price)

X_train_1,X_test_1,y_train_1,y_test_1=train_test_split(X_car_price,y_car_price,test_size=0.2,random_state=42)
print("x train: ",X_train_1.shape)
print("x test: ",X_test_1.shape)
print("y train: ",y_train_1.shape)
print("y test: ",y_test_1.shape)


# Breast cancer classification 
```
(Classification problem [second dataset])
```


In [ ]:
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data,columns=cancer.feature_names)
df.head()

In [ ]:
df.columns

In [ ]:
print(df.shape)
print(df.isnull().sum())

In [ ]:
X = df
y = pd.Series(cancer.target)
X_train_2, X_test_2,y_train_2, y_test_2= train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)

X_train_2=np.array(X_train_2)
X_test_2=np.array(X_test_2)
y_train_2=np.array(y_train_2)
y_test_2=np.array(y_test_2)
y_train_2 = to_categorical(y_train_2, dtype ="uint8")
y_test_2 = to_categorical(y_test_2, dtype ="uint8")
print('X_train: ' + str(X_train_2.shape))
print('Y_train: ' + str(y_train_2.shape))
print('X_test:  '  + str(X_test_2.shape))
print('Y_test:  '  + str(y_test_2.shape))

# Neural Network Implementation

## Implementing Layer class

In [ ]:
class Input_Layer:
    def __init__(self,features,data):
        self.neurons=features
        self.input=data
class Layer:
    def __init__(self,n_input,n_neurons):
        self.neurons=n_neurons
        self.weights=(np.random.rand(n_input,n_neurons)-0.5)*sqrt(1/(n_input+n_neurons))
    def forward(self,inputs):
        self.input=inputs
        self.output=np.dot(inputs,self.weights)
    def backward(self,dvalues):
        inputs=self.input.reshape(-1, 1)
        self.dweights =  np.dot(inputs, dvalues)
        # Gradient on values
        self.dinputs  = np.dot(dvalues, self.weights.T)

## Implementing Activation functions to be used in NN


In [ ]:
class Linear :
    def forward (self ,inputs):
        self.inputs = inputs
        self.output  = inputs
    def backward(self, dvalues):
        self.dinputs  =  dvalues.copy()
        
class Sigmoid:
    def forward(self,inputs):
        inputs = np.float128(inputs)
        self.output=(1/(1+exp(-inputs)))
    def backward(self,dvalues=None):
        self.dinputs =(1 - self.output) * self.output
        
class ReLU:
    def forward(self,inputs):
        self.output=np.maximum(0,inputs)
    def backward(self,outputs):
        self.dinputs = outputs.copy()
        self.dinputs[self.output <= 0] = 0
        
class Softmax:
    def forward(self,inputs):
        z=exp(inputs-np.max(inputs,axis=1,keepdims=True))
        probs=z/np.sum(z,axis=1,keepdims=True)
        self.output=probs
    def backward(self,dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) -np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)

## Loss Functions Implementation

## Regression loss Mean Sqaure Error (MSE)

In [ ]:
class MSE:
    def forward(self,y,y_hat):
        self.output=mean((y_hat-y)**2,axis=-1)
        self.error=mean((y_hat-y)**2)
    def backward(self,dvals,y):
        samples = len(dvals)
        # outputs = len(dvals[0])
        self.dinputs = -2 * (y - dvals) / samples
        self.dinputs = self.dinputs / samples

## Regression structure

In [ ]:
n_neurons=20
eta=0.07
epochs=5000
#input layer
layer0=Input_Layer(X_train_1.shape[1],X_train_1)
#hidden layer
layer1=Layer(layer0.neurons,n_neurons)
activation1=Sigmoid()
#output layer
layer2=Layer(layer1.neurons,1)
activation2=Linear()
for epoch in range(epochs):
    y_pred=[]
    for x,y in zip(X_train_1,y_train_1):
        #Forward Path
        layer1.forward(x)
        activation1.forward(layer1.output)
        layer2.forward(activation1.output)
        activation2.forward(layer2.output)
        y_pred.append(activation2.output)
        loss=MSE()
        loss.forward(y,activation2.output)
        #backward Path
        activation2.backward(layer2.output)
        dinputs2=activation2.dinputs
        layer2.backward(dinputs2)
        activation1.backward(layer1.output)
        dinputs1=activation1.dinputs.reshape(-1,1)
        layer1.backward(dinputs1.T)
        layer1.weights += (-eta* layer1.dweights)*0.5
        layer2.dweights=layer2.dweights.reshape(-1,1)
        layer2.weights += (-eta* layer2.dweights)
        layer1.weights=(layer1.weights-min(layer1.weights))/(max(layer1.weights)-min(layer1.weights))
        layer2.weights=(layer2.weights-min(layer2.weights))/(max(layer2.weights)-min(layer2.weights))
    if epoch%500==0:
        y_pred=np.array(y_pred)
        loss_=MSE()
        loss_.forward(y_train_1,y_pred)
        print('Loss in epoch{}'.format(epoch),loss_.error)
    if epoch%1000==0:
        print('layer1.weights',layer1.weights)                        
        print('layer2.weights',layer2.weights)

In [ ]:
weights1=layer1.weights
weights2=layer2.weights

In [ ]:
def predict(X,layer1,layer2,activation1,activation2):
    y_pred=[]
    for x in X:
        layer1.forward(x)
        activation1.forward(layer1.output)
        layer2.forward(activation1.output)
        activation2.forward(layer2.output)
        y_pred.append(activation2.output)
    return np.array(y_pred)
def accuracy(y,y_hat,regression=True):
    if regression:
        loss=MSE()
        loss.forward(y,y_hat)
        return loss.error
    else:
        length=y.shape[0]
        right=0
        for y,y_hat in zip(y,y_pred):
            if (y==y_hat ).all():
                rights+=1
        return right/length

In [ ]:
y_pred_1=predict(X_test_1,layer1,layer2,activation1,activation2)
acc_1=accuracy(y_test_1,y_pred_1)
acc_1

## Classification Arch

In [ ]:
class Sigmoid:
    def forward(self,inputs):
        inputs = np.float128(inputs)
        self.output=(1/(1+exp(-inputs)))
    def backward(self,dvalues=None):
        self.dinputs =(1 - self.output) * self.output
        
class ReLU:
    def forward(self,inputs):
        self.output=np.maximum(0,inputs)
    def backward(self,outputs):
        self.dinputs = outputs.copy()
        self.dinputs[self.output <= 0] = 0
        
class Softmax:
    def forward(self,inputs):
        z=exp(inputs-np.max(inputs,axis=1,keepdims=True))
        probs=z/np.sum(z,axis=1,keepdims=True)
        self.output=probs
    def backward(self,dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) -np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)

In [ ]:
# Binary cross-entropy loss
class Loss_BinaryCrossentropy:
    def forward (self,y_pred,y_true ):
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = clip(y_pred,1e-7,1-1e-7 )
        # Calculate sample-wise loss
        sample_losses  = - (y_true *log(y_pred_clipped) +(1-y_true)* log( 1 - y_pred_clipped))
        sample_losses  = mean(sample_losses,  axis =- 1 )
        self.error=sample_losses
        return sample_losses
    def backward(  self , dvalues,y_true ):
        samples = len(dvalues)
        outputs =  len (dvalues[0])
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        clipped_dvalues =clip(dvalues, 1e-7, 1  - 1e-7 )
        # Calculate gradient
        self.dinputs = -(y_true /clipped_dvalues -( 1 - y_true)/(  1  -  clipped_dvalues))/outputs
        # Normalize gradient
        self.dinputs =  self.dinputs/samples

In [ ]:
n_neurons=20
eta=0.01
epochs=5000
#input layer
layer0_0=Input_Layer(X_train_2.shape[1],X_train_2)
#hidden layer
layer1_1=Layer(layer0_0.neurons,n_neurons)
activation1_1=Sigmoid()
#output layer
layer2_1=Layer(layer1.neurons,2)
activation2_1=Sigmoid()
for epoch in range(epochs):
    y_pred=[]
    for x,y in zip(X_train_2,y_train_2):
        #Forward Path
        layer1_1.forward(x)
        activation1_1.forward(layer1_1.output)
        layer2_1.forward(activation1_1.output)
        activation2_1.forward(layer2_1.output)
        y_pred.append(activation2_1.output)
        loss=Loss_BinaryCrossentropy()
        loss.forward(activation2_1.output,y)
        #backward Path
        activation2_1.backward(layer2_1.output)
        dinputs2=activation2_1.dinputs.reshape(-1,1)
        layer2_1.backward(dinputs2.T)
        activation1_1.backward(layer1_1.output)
        dinputs1=activation1_1.dinputs.reshape(-1,1)
        layer1_1.backward(dinputs1.T)
        
        layer1_1.weights += (-eta* layer1_1.dweights)*0.5
        # layer2_1.dweights=layer2_1.dweights.reshape(-1,1)
        layer2_1.weights += (-eta* layer2_1.dweights)
        layer1_1.weights = layer1_1.weights/ layer1_1.weights.max() 
        layer2_1.weights = layer2_1.weights/ layer2_1.weights.max()
        
    if epoch%500==0:
        print(y_pred)
        y_pred=np.array(y_pred)
        y_pred=np.ceil(y_pred)
        loss_=Loss_BinaryCrossentropy()
        loss_.forward(y_pred,y_train_2)
        # print(y_pred)
        print('Loss in epoch{}'.format(epoch),loss_.error)
    # if epoch%1000==0:
    #     print('layer1.weights',layer1_1.weights)                        
    #     print('layer2_1.weights',layer2_1.weights)

In [ ]:
y_pred_2=predict(X_test_2,layer1_1,layer2_1,activation1_1,activation2_1)
y_pred_2=np.ceil(y_pred_2)

acc_2=accuracy(y_test_2,y_pred_2,regression=False)
acc_2